# **SQLite 연습문제**

## **연습문제 1**

투수들의 기록중에서 평균자책점(ERA), 투구인닝(IP), 탈삼진(SO) 기록을 찾아서 Pitcher_stats 란 테이블을 만들고, Eagles 테이블과 Join 하여 백넘버, 선수명, 포지션, 투구인닝, 평균자책점, 탈삼진 필드를 갖는 데이터 프레임을 만들어서 Join 한 결과를 입력하고, 그 결과를 보이시오.

### **1.1 데이터 베이스 접속**

In [1]:
import sqlite3

conn = sqlite3.connect('./test.db') 

### **1.2 DB에 Pitcher_stats 테이블 생성 준비**

In [2]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Pitcher_stats \
    (back_no INT NOT NULL, \
     player TEXT, \
     ERA INT, \
     IP REAL, \
     SO INT, \
     PRIMARY KEY(back_no));')

### **1.3 pitcher.csv 파일을 pitcher로 저장**

In [3]:
import pandas as pd

pitcher = pd.read_csv('./pitcher.csv', encoding='EUC-KR')
pitcher

,back_no,player,ERA,IP,SO
0,17,김범수,5.67,81.00,65
1,38,안영명,3.43,44.67,35
2,36,장민재,4.81,91.67,84


### **1.4 pitcher 데이터프레임에 있는 데이터를 DB의 Pitcher_stats 테이블에 할당**

In [4]:
# 앞에서 생성된 Pitcher_stats에 데이터 할당

cur = conn.cursor() 
sql = 'INSERT INTO Pitcher_stats VALUES (?, ?, ?, ?, ?);'

for i in range(3):
    cur.execute(sql, (int(pitcher.iloc[i, 0]),
                      pitcher.iloc[i, 1],
                      float(pitcher.iloc[i, 2]),
                      float(pitcher.iloc[i, 3]),
                      int(pitcher.iloc[i, 4])
                      ))
conn.commit()

### **1.5 SQL문으로 Eagles 테이블과 Pitcher_stats 테이블 Join**

In [5]:
# Eagles와 pitcher 테이블 join

join = "SELECT e.back_no, e.name, e.position, \
        p.ERA, p.IP, p.SO \
        FROM Eagles AS e JOIN Pitcher_stats AS p \
        ON e.name like p.player;"

cur = conn.cursor()
cur.execute(join)

pitcher_join = cur.fetchall()
pitcher_join

[(17, '김범수', '투수', 5.67, 81.0, 65),
 (38, '안영명', '투수', 3.43, 44.67, 35),
 (36, '장민재', '투수', 4.81, 91.67, 84)]

### **1.6 Join한 결과를 데이터프레임으로 변환**

In [6]:
# join한 결과를 데이터프레임으로 변환

colName = ['등번호', '선수명', '포지션', '방어율', '투구이닝', '탈삼진']  
pitcher_df = pd.DataFrame(pitcher_join, columns = colName)
pitcher_df

,등번호,선수명,포지션,방어율,투구이닝,탈삼진
0,17,김범수,투수,5.67,81.00,65
1,38,안영명,투수,3.43,44.67,35
2,36,장민재,투수,4.81,91.67,84


In [85]:
conn.close()

## **연습문제 2**

다음의 지시대로 DB 테이블을 만들고 이를 조회하는 프로그램을 만드시오.  

1. 국내의 대표적인 걸그룹 또는 보이그룹 5개 이상에 대하여 다음과 같은 정보를 갖는 테이블을 만드시오. 

    - id(Primary Key). group_name, member_num, debut_date, company


2. 이들이 불렀던 노래 또는 다른 사람이 불렀던 노래 10곡 이상에 대하여 다음의 정보를 갖는 테이블을 만드시오.

    - song_id(Primary_Key), song_name, group_id, am_year, songwriter, intro_lyrics  
        

3. 위 두 개의 테이블을 Join한 결과를 가지고 다음의 필드를 갖는 데이터프레임을 만드시오.

    - group_name, member_num, debut_date, song_name, am_year

### **2.1 DB에 Group_1 테이블 및 Group_2 테이블 생성**

#### **2.1.1 Group_1 테이블 생성**

In [1]:
import sqlite3

conn = sqlite3.connect('./test.db') 

In [2]:
# groups_1 테이블 생성 준비 

cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS groups_1 \
    (id INT NOT NULL, \
     group_name TEXT, \
     member_num INT, \
     debut_date DATE, \
     company TEXT, \
     PRIMARY KEY(id));')

In [3]:
import pandas as pd

groups_1 = pd.read_csv('./groups_1.csv', encoding='EUC-KR')
groups_1

,id,group_name,member_num,debut_date,company
0,1001,블랙핑크,4,2016-08-08,YG
1,1002,트와이스,9,2015-10-20,JYP
2,1003,ITZY,5,2019-02-12,JYP
3,1004,마마무,4,2014-06-19,RBW
4,1005,여자친구,6,2015-01-15,쏘스뮤직


In [4]:
# groups_1 테이블에 데이터 할당 

cur = conn.cursor() 
sql = 'INSERT INTO groups_1 VALUES (?, ?, ?, ?, ?);'

for i in range(5):
    cur.execute(sql, (int(groups_1.iloc[i, 0]),
                      groups_1.iloc[i, 1],
                      int(groups_1.iloc[i, 2]),
                      groups_1.iloc[i, 3],
                      groups_1.iloc[i, 4]
                      ))
conn.commit()

#### **2.1.2 Group_2 테이블 생성**

In [2]:
# groups_2 테이블 생성 준비 

cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS groups_2 \
    (song_id INT NOT NULL, \
     song_name TEXT, \
     group_id TEXT, \
     am_year DATE, \
     songwrite TEXT, \
     intro_lyrics, \
     PRIMARY KEY(song_id));')

In [3]:
import pandas as pd

groups_2 = pd.read_csv('./groups_2.csv', encoding='EUC-KR')
groups_2

,song_id,song_name,group_id,am_date,songwriter,intro_lyrics
0,1001,Kill this love,블랙핑크,2009-04-05,TEDDV,천사 같은 Hi 끝엔 악마 같은 Bye
1,1003,달라달라,ITZY,2009-02-12,별들의전쟁,DDA-DDA-LA-DDA-LA-DDA-LA
2,1004,고고베베,마마무,2009-03-14,김건모,일루와 일루와 일루와 일루와
3,1007,Loca,FAVORITE,2009-01-15,오종훈,힘풀려 널불러 눈풀려 널불러
4,1009,그 시절 우리가 사랑했던 우리,러블리즈,2009-05-20,스페이스카우보이,아직도 펼쳐보면 웃음이 나와


In [4]:
# groups_2 테이블에 데이터 할당 

cur = conn.cursor() 
sql = 'INSERT INTO groups_2 VALUES (?, ?, ?, ?, ?, ?);'

for i in range(5):
    cur.execute(sql, (int(groups_2.iloc[i, 0]),
                      groups_2.iloc[i, 1],
                      groups_2.iloc[i, 2],
                      groups_2.iloc[i, 3],
                      groups_2.iloc[i, 4],
                      groups_2.iloc[i, 5]
                      ))
conn.commit()

### **2.2 Groups_1 테이블과 Groups_2 테이블 Join**

In [5]:
# groups_1, groups_2 table join
# 노래 발표일자 순으로 하려면 ORDER BY two.am_date 를 추가해준다.

join = "SELECT one.id, one.group_name, one.member_num, one.debut_date, \
        two.song_name, two.am_year \
        FROM groups_1 AS one JOIN groups_2 AS two \
        ON one.id like two.song_id;"

cur = conn.cursor()
cur.execute(join)

groups_join = cur.fetchall()  # 전체 조회
groups_join

[(1001, '블랙핑크', 4, '2016-08-08', 'Kill this love', '2009-04-05'),
 (1003, 'ITZY', 5, '2019-02-12', '달라달라', '2009-02-12'),
 (1004, '마마무', 4, '2014-06-19', '고고베베', '2009-03-14')]

### **2.3 Join한 결과를 데이터프레임으로 변환**

In [7]:
# join한 결과를 데이터프레임으로 변환

colName = ['id', '그룹명', '멤버수', '데뷔일', '곡 명', '곡 발표일']  
Groups_df = pd.DataFrame(groups_join, columns = colName)
Groups_df

,id,그룹명,멤버수,데뷔일,곡 명,곡 발표일
0,1001,블랙핑크,4,2016-08-08,Kill this love,2009-04-05
1,1003,ITZY,5,2019-02-12,달라달라,2009-02-12
2,1004,마마무,4,2014-06-19,고고베베,2009-03-14


## **연습문제 3**

사용자의 이름과, 비밀번호를 갖는 Users 테이블이 있다.
사용자의 이름과 비밀번호를 올바르게 입력하면 ‘성공’을 출력하고, 잘못 입력하면 ‘실패’를 출력하는 프로그램을 작성하시오.

### **3.1 DB 테이블 생성 준비**

In [2]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS login \
    (id TEXT, \
     password INT NOT NULL, \
     PRIMARY KEY(id));')

### **3.2 DB에 할당할 데이터프레임 생성**

In [3]:
import pandas as pd

data = {
    'id' : ['aaa1234', 'abc1549', 'soso999'],
    'password' : ['dlwps', 'zjavbxj', 'gkrdnjs']
}

Login_df = pd.DataFrame(data, columns = data.keys())
Login_df

,id,password
0,aaa1234,dlwps
1,abc1549,zjavbxj
2,soso999,gkrdnjs


### **3.3 DB테이블에 데이터 할당**

In [4]:
cur = conn.cursor() 
sql = 'INSERT INTO login VALUES (?, ?);'

for i in range(3):
    cur.execute(sql, (Login_df.iloc[i, 0],
                      Login_df.iloc[i, 1]
                      ))
conn.commit()

### **3.4 DB테이블을 불러들인 후 데이터프레임 조회**

In [18]:
login_df_DB = "SELECT l.id, l.password FROM login AS l;"

cur = conn.cursor()
cur.execute(login_df_DB)

login_all = cur.fetchall()  # 전체 조회
login_db_system = pd.DataFrame(login_all, columns = ['id', 'password'])
login_db_system

,id,password
0,aaa1234,dlwps
1,abc1549,zjavbxj
2,soso999,gkrdnjs


### **3.5 간단한 로그인 함수 만들기**

In [73]:
def login(uid, upw):
    is_correct = '로그인에 실패했습니다.'
    
    for i in range(len(login_db_system)):
        if uid == login_db_system.iloc[i, 0]:
            if upw == login_db_system.iloc[i, 1]:
                is_correct = '로그인에 성공했습니다.'
            elif upw != login_db_system.iloc[i, 1]:
                is_correct = '비밀번호가 틀렸습니다.'
            else:
                is_correct = '아이디가 틀렸습니다.'
            
    return is_correct

In [77]:
login('aaa1234', 'gkrdnjs')

'비밀번호가 틀렸습니다.'

## **교수님 모범답안**

In [8]:
cur = conn.cursor()
cur.execute("SELECT * FROM login;")

rows = cur.fetchall()

for row in rows:
    print(row)

('aaa1234', 'dlwps')
('abc1549', 'zjavbxj')
('soso999', 'gkrdnjs')


### **예외 처리 하기 전**

In [13]:
def checkUser(uid, password):
    print(uid, password)
    conn = sqlite3.connect('./test.db')
    cur = conn.cursor()
    sql = "SELECT id, password FROM login WHERE id like ?;"
    cur.execute(sql, (uid,))
    dbname, dbpassword = cur.fetchone()
    print(dbname, dbpassword)
    
    conn.close()

In [14]:
checkUser('aaa1234', 'dlwps')

aaa1234 dlwps
aaa1234 dlwps


### **모든 예외를 처리하는 코드**